# Polynomial Variable Regression
* 다항 회귀는 비선형 데이터를 선형 모델로 학습할 수 있다.
* 각 특성(변수)의 제곱한 새로운 특성을 추가하고 선형 모델로 학습한다.
* 2차식을 포함한 데이터를 생성해서 실험한다.
* [np.polyval(p, x)](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.polyval.html): 함수는 x에 대해서 다항식 계수를 갖는 p의 y값을 생성한다.
    * p는 원소의 갯수에 따라 0차, 1차, 2차식의 계수를 나타낸다.
    * p = np.array([2,3,5]) : $2x^2 + 3x + 5$

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pylab as plt

m = 100
X = 5 * np.random.rand(m, 1) -3
y = 2*X**2 + 3*X + 5 + np.random.randn(m, 1) # 2,3,5 +noiseb
print(X.shape, y.shape, X[0], y[0])
plt.plot(X, y, '.')
#xl = np.linspace(-3, 2, m)
#yl = np.polyval([2,3,5], xl)
#plt.plot(xl, yl, 'r-')

### 다항식을 다변수식으로 변환
* X에 제곱을 한 새로운 항을 추가하고 bias에 해당하는 1항을 추가
    * 차수를 높인 항을 추가
* 추가한 항의 값을 모두 더해서 표시하면 선형 형태를 띈다. 
* 이 모델을 다변수 회귀 모델 훈련에 사용한 방법을 그대로 적용
* 다차항을 계속 추가 하면 과유연한 또는 과적합되는 모델을 얻을 수 있다.

In [ ]:
degree = 3 # can change the degree higher
#X_poly = np.concatenate( (X**2, X, np.ones((m,1))),axis= 1)  # same with the code below 
X_poly = np.vander(X.ravel(), degree)
print(X_poly.shape, X_poly[0])
plt.plot(X_poly.sum(axis=1), y, '.')
p1 = np.polyfit(X_poly.sum(axis=1), y,1)
print(p1)
plt.plot(X_poly.sum(axis=1), np.polyval(p1, X_poly.sum(axis=1)))


In [ ]:
W = np.random.rand(degree,1)
print('initial W', W.ravel())

learning_rate = 5e-4#0.0005
epochs = 30000
for epoch in range(epochs):
    hypothesis = X_poly.dot(W)
    loss = hypothesis - y
    cost = 1/m * np.sum(loss**2)
    gradient = 2/m * X_poly.T.dot(loss)
    W = W - learning_rate * gradient
    if epoch %1000 == 0:
        print('epoch:',epoch,' cost:',cost)
print('Final W:',W.ravel(), 'cost:%f'%cost, 'gradient:', gradient.ravel())

plt.plot(X, y, '.')
xl = np.linspace(X.min(), X.max(), m)
yl = np.polyval(W.ravel(), xl)
plt.plot(xl, yl, 'r-')


## scikit-learn 구현


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

plt.plot(X, y, '.')

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly2 = poly_features.fit_transform(X)
print(X[0], X_poly2[0])

lin_reg = LinearRegression()
lin_reg.fit(X_poly2, y)
print(lin_reg.intercept_, lin_reg.coef_)

X_new=np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)
plt.plot(X_new, y_new, "r-", linewidth=2, label="예측")
plt.show()